In [8]:
# !pip install opencv-python
# !pip install tensorflow
!pip install --upgrade tensorflow-hub

     -------------------------------------- 100.6/100.6 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import tensorflow as tf
import numpy as np

In [16]:
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (640, 640))
    image = image / 255.0
    image = np.expand_dims(image, axis=0).astype('float32')S
    return image

In [15]:
def postprocess(result, original_image):
    num_detections = int(result.pop('num_detections'))
    detection_boxes = result['detection_boxes'][:num_detections]
    detection_classes = result['detection_classes'][:num_detections].astype(int)
    detection_scores = result['detection_scores'][:num_detections]
    image_height, image_width, _ = original_image.shape
    detection_boxes = detection_boxes * np.array([image_height, image_width, image_height, image_width])
    final_boxes = []
    final_labels = []
    for i in range(num_detections):
        if detection_scores[i] > 0.5:  
            box = detection_boxes[i].astype(int)
            label = detection_classes[i]
            final_boxes.append(box)
            final_labels.append(label)
    return final_boxes, final_labels

In [9]:
import tensorflow as tf
import tensorflow_hub as hub

# Specify the model's URL from TensorFlow Hub
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# Load the model
detector = hub.load(module_handle).signatures['default']

# Save the model locally
tf.saved_model.save(detector, "./model/")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


In [17]:
# Load the model
model = tf.saved_model.load('./model/')
# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame
    input_frame = preprocess(frame)

    # Run the model
    outputs = model(input_frame)

    # Postprocess the outputs
    boxes, labels = postprocess(outputs, frame)

    # Draw the bounding boxes on the original frame
    for box, label in zip(boxes, labels):
        cv2.rectangle(frame, (box[1], box[0]), (box[3], box[2]), (255, 0, 0), 2)
        cv2.putText(frame, str(label), (box[1], box[0]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Webcam Stream', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

TypeError: pruned(images): expected argument #0(zero-based) to be a Tensor; got ndarray ([[[[0.4509804  0.54901963 0.5647059 ]
   [0.44705883 0.5529412  0.56078434]
   [0.43529412 0.54901963 0.5568628 ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.44705883 0.54509807 0.56078434]
   [0.44313726 0.54901963 0.5568628 ]
   [0.4392157  0.54509807 0.5568628 ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.4509804  0.54901963 0.5529412 ]
   [0.44313726 0.54509807 0.5529412 ]
   [0.4392157  0.5411765  0.5529412 ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[0.22352941 0.34117648 0.40392157]
   [0.23137255 0.34509805 0.40784314]
   [0.23529412 0.34117648 0.40784314]
   ...
   [0.56078434 0.70980394 0.80784315]
   [0.5568628  0.7058824  0.807843